In [56]:
import time
import pandas as pd
 

from bs4 import BeautifulSoup
import urllib.request as req

import requests
import os
import datetime
import re
import glob

from team_index import get_teams

now = datetime.datetime.now()

os.chdir('/Users/sasno/Desktop/MyPandas/vleague')

headers = {"User-Agent": "Mozilla/5.0"}

# print('ディビジョンを選択してください:{v1_m, v2_m, v3_m, v1_w, v2_w}')
# division = input()
division = 'v1_m'
teams = get_teams(division)

if not os.path.isdir(division):
    os.makedirs(division)
os.chdir(division)

if not os.path.isdir('all'):
    os.makedirs('all')
os.chdir('all')

career_dir = 'career'
# print(career_dir)
if not os.path.isdir(career_dir):
    os.makedirs(career_dir)
    os.chdir(career_dir)

# print(os.getcwd())

career_lists = []

for team_id, team_name in teams.items():
    url = 'https://www.vleague.jp/record/team_players/{}'.format(team_id)
    # print(url)
    request = req.Request(url, headers=headers)
    response = req.urlopen(request)
    parse_html = BeautifulSoup(response, 'html.parser')

    tables = parse_html.find_all('table')[0]
    tr = tables.find_all('tr')
    a = tables.find_all('a')
    player_urls = []
    for i in a:
        href = i.attrs['href']
        # print(href)
        player_url = 'https://www.vleague.jp{}'.format(href)
        # print(player_url)
        player_urls.append(player_url)
    
    number_list = []
    player_list = []
    for i in tr[1:]:
        td = i.find_all('td')
        number = td[0].text
        number_list.append(number)
        player = td[1].text.replace('\xa0', '')
        player_list.append(player)
    # print(number_list, player_list)


    # print(os.getcwd())

    for i in range(len(player_list)):
        html = requests.get(player_urls[i], headers=headers)
        request_2 = req.Request(player_urls[i], headers=headers)
        response_2 = req.urlopen(request_2)
        parse_html_2 = BeautifulSoup(response_2, 'html.parser')
        table_2 = parse_html_2.find_all('table')

        if len(table_2) > 0: 
            data = pd.read_html(html.text, header = [1])
            stats = pd.DataFrame(data[0])
            # print(stats)
            stats = stats.rename({"チ｜ム": "チーム", "サ｜ブ": "サーブ", "ノ｜タッチ": "ノータッチ", "エ｜ス": "エース"}, axis=1)
            stats = stats.rename(columns={'打数':'アタック打数', '得点':'アタック得点', 
                            '失点':'アタック失点', '決定率':'アタック決定率',
                            'セ平ット均':'アタックセット平均','打数.1':'バックアタック打数', '得点.1':'バックアタック得点',
                            '失点.1':'バックアタック失点', '決定率.1':'バックアタック決定率', 
                            'セット平均':'アタック得点セット平均' ,'得点.2':'ブロック得点', 
                            'セット平均.1':'ブロック得点セット平均','セ平ット均.1':'ブロックセット平均', 
                            '打数.2':'サーブ打数', '得点.3':'サーブ得点', '失点.2':'サーブ失点',
                            '効果':'サーブ効果', '効果率':'サーブ効果率', '成功・優':'サーブレシーブ成功・優', 
                            '成功・良':'サーブレシーブ成功・良', '成功率':'サーブレシーブ成功率'})
            stats.insert(2, '選手名', player_list[i])
            stats.insert(9, 'アタック効果率', (((stats['アタック得点'] - stats['アタック失点']) / stats['アタック打数']) * 100).round(1))
            stats.insert(14, 'バックアタック効果率', (((stats['バックアタック得点'] - stats['バックアタック失点']) / stats['バックアタック打数']) * 100).round(1))
            career_lists.append(stats)
            career_database = pd.concat(career_lists, ignore_index=True)
            # csv = '{0}_{1}_{2:%Y}-{2:%m}-{2:%d}.csv'.format(number_list[i], player_list[i], now)
            # stats.to_csv(csv, index=False, encoding='cp932')
            # print('{}を作成'.format(csv))
career_database

,大会,チーム,選手名,試合数,セット数,アタック打数,アタック得点,アタック失点,アタック決定率,アタック効果率,...,サーブ効果,サーブ効果率,受数,サーブレシーブ成功・優,サーブレシーブ成功・良,サーブレシーブ成功率,アタック,ブロック,サーブ,得点合計
0,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,藤中優斗,29,79,152,64,12,42.1,34.2,...,41,4.1,287,120,85,56.6,64,14,5,83
1,レギュラーラウンド,ジェイテクトSTINGS,藤中優斗,27,70,129,53,8,41.1,34.9,...,37,4.5,224,93,67,56.5,53,13,5,71
2,セミファイナル（３位決定戦）,ジェイテクトSTINGS,藤中優斗,1,4,10,5,2,50.0,30.0,...,2,1.6,35,17,9,61.4,5,1,0,6
3,ファイナル,ジェイテクトSTINGS,藤中優斗,1,5,13,6,2,46.2,30.8,...,2,0.0,28,10,9,51.8,6,0,0,6
4,V.LEAGUE DIVISION1 MEN 2018-19,ジェイテクトSTINGS,藤中優斗,4,6,0,0,0,-,NaN,...,0,-,5,3,1,70,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2951,レギュラーラウンド,VC長野トライデンツ,長田翼,21,28,0,0,0,-,NaN,...,12,7.7,0,0,0,-,0,0,0,0
2952,Ｖ・チャレンジリーグⅡ男子 2015/16,VC長野トライデンツ,長田翼,3,5,0,0,0,-,NaN,...,1,3.6,0,0,0,-,0,0,0,0
2953,レギュラーラウンド,VC長野トライデンツ,長田翼,3,5,0,0,0,-,NaN,...,1,3.6,0,0,0,-,0,0,0,0
2954,レギュラーシーズン通算,NaN,長田翼,99,177,11,7,2,63.6,45.5,...,71,5.9,8,0,0,0.0,7,12,8,27


In [112]:
career_database = career_database.replace('-', None)
career_database

,大会,チーム,選手名,試合数,セット数,アタック打数,アタック得点,アタック失点,アタック決定率,アタック効果率,...,サーブ効果,サーブ効果率,受数,サーブレシーブ成功・優,サーブレシーブ成功・良,サーブレシーブ成功率,アタック,ブロック,サーブ,得点合計
0,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,藤中優斗,29,79,152,64,12,42.1,34.2,...,41,4.1,287,120,85,56.6,64,14,5,83
1,レギュラーラウンド,ジェイテクトSTINGS,藤中優斗,27,70,129,53,8,41.1,34.9,...,37,4.5,224,93,67,56.5,53,13,5,71
2,セミファイナル（３位決定戦）,ジェイテクトSTINGS,藤中優斗,1,4,10,5,2,50.0,30.0,...,2,1.6,35,17,9,61.4,5,1,0,6
3,ファイナル,ジェイテクトSTINGS,藤中優斗,1,5,13,6,2,46.2,30.8,...,2,0.0,28,10,9,51.8,6,0,0,6
4,V.LEAGUE DIVISION1 MEN 2018-19,ジェイテクトSTINGS,藤中優斗,4,6,0,0,0,46.2,NaN,...,0,0.0,5,3,1,70,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2951,レギュラーラウンド,VC長野トライデンツ,長田翼,21,28,0,0,0,80.0,NaN,...,12,7.7,0,0,0,0.0,0,0,0,0
2952,Ｖ・チャレンジリーグⅡ男子 2015/16,VC長野トライデンツ,長田翼,3,5,0,0,0,80.0,NaN,...,1,3.6,0,0,0,0.0,0,0,0,0
2953,レギュラーラウンド,VC長野トライデンツ,長田翼,3,5,0,0,0,80.0,NaN,...,1,3.6,0,0,0,0.0,0,0,0,0
2954,レギュラーシーズン通算,NaN,長田翼,99,177,11,7,2,63.6,45.5,...,71,5.9,8,0,0,0.0,7,12,8,27


In [214]:
regular19 = career_database[career_database['大会'] == '2019-20 V.LEAGUE DIVISION1 MEN']
regular19

,大会,チーム,選手名,試合数,セット数,アタック打数,アタック得点,アタック失点,アタック決定率,アタック効果率,...,サーブ効果,サーブ効果率,受数,サーブレシーブ成功・優,サーブレシーブ成功・良,サーブレシーブ成功率,アタック,ブロック,サーブ,得点合計
0,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,藤中優斗,29,79,152,64,12,42.1,34.2,...,41,4.1,287,120,85,56.6,64,14,5,83
8,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,金丸晃大,27,51,17,8,1,47.1,41.2,...,8,4.3,2,0,1,25.0,8,5,0,13
60,2019-20 V.LEAGUE DIVISION1 MEN,東レアローズ,伏見大和,56,86,135,84,7,62.2,57.0,...,27,3.2,5,3,2,80.0,84,36,9,129
89,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,福山汰一,29,47,94,55,2,58.5,56.4,...,30,6.0,8,3,3,56.2,55,21,9,85
108,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,饒書涵,27,96,296,177,7,59.8,57.4,...,84,6.9,25,12,7,62,177,40,7,224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2910,2019-20 V.LEAGUE DIVISION1 MEN,VC長野トライデンツ,森崎健史,27,91,217,112,19,51.6,42.9,...,62,3.9,53,28,11,63.2,112,30,11,153
2922,2019-20 V.LEAGUE DIVISION1 MEN,VC長野トライデンツ,松村文斗,24,3,10,1,2,10,-10.0,...,1,-10,21,8,7,54.8,1,0,0,1
2930,2019-20 V.LEAGUE DIVISION1 MEN,VC長野トライデンツ,大沢健太郎,22,17,0,0,0,35.6,NaN,...,0,3,19,4,10,47.4,0,0,0,0
2936,2019-20 V.LEAGUE DIVISION1 MEN,VC長野トライデンツ,高澤和貴,27,62,14,8,1,57.1,50.0,...,6,9.2,67,30,19,59,8,0,4,12


In [215]:
regular19['アタック決定率'] = regular19['アタック決定率'].astype(float)
regular19['バックアタック決定率'] = regular19['バックアタック決定率'].astype(float)
regular19['アタック得点セット平均'] = regular19['アタック得点セット平均'].astype(float)
regular19['ブロック得点セット平均'] = regular19['ブロック得点セット平均'].astype(float)
regular19['サーブ効果率'] = regular19['サーブ効果率'].astype(float)
regular19['サーブレシーブ成功率'] = regular19['サーブレシーブ成功率'].astype(float)

In [216]:
regular19.dtypes

大会              object
チーム             object
選手名             object
試合数              int64
セット数             int64
アタック打数           int64
アタック得点           int64
アタック失点           int64
アタック決定率        float64
アタック効果率        float64
バックアタック打数        int64
バックアタック得点        int64
バックアタック失点        int64
バックアタック決定率     float64
バックアタック効果率     float64
アタック得点セット平均    float64
ブロック得点           int64
ブロック得点セット平均    float64
サーブ打数            int64
ノータッチ            int64
エース              int64
サーブ失点            int64
サーブ効果            int64
サーブ効果率         float64
受数               int64
サーブレシーブ成功・優      int64
サーブレシーブ成功・良      int64
サーブレシーブ成功率     float64
アタック             int64
ブロック             int64
サーブ              int64
得点合計             int64
dtype: object

In [130]:
 ob_data = regular19[0:1].to_dict()
 print(ob_data['大会'][0])

2019-20 V.LEAGUE DIVISION1 MEN


In [222]:
attack_ave = (regular19['アタック得点'].sum() / regular19['アタック打数'].sum()) * 100
attack_ef = ((regular19['アタック得点'].sum() - regular19['アタック失点'].sum()) / regular19['アタック打数'].sum()) * 100
b_attack_ave = (regular19['バックアタック得点'].sum() / regular19['バックアタック打数'].sum()) * 100
b_attack_ef = ((regular19['バックアタック得点'].sum() - regular19['バックアタック失点'].sum()) \
        / regular19['バックアタック打数'].sum()) * 100
attack_by_set = regular19['アタック得点'].sum() / regular19['セット数'].sum()
block_by_set = regular19['ブロック得点'].sum() / regular19['セット数'].sum()
serve_ef = ((regular19['ノータッチ'].sum() + regular19['エース'].sum()) * 100 \
        + regular19['サーブ効果'].sum() * 25 - regular19['サーブ失点'].sum() * 25) \
        / regular19['サーブ打数'].sum()
s_receive_ave = \
     (regular19['サーブレシーブ成功・優'].sum() * 100 + \
     regular19['サーブレシーブ成功・良'].sum() * 50) \
     / regular19['受数'].sum()

In [159]:
# value = regular19.columns.values
# print('regular19.append(')
# for i in range(len(value)):
#     if i <= 2:
#         print("     ob_data['{}'][0],".format(value[i]))
#     else:
#         print("     regular19['{}'].sum(),".format(value[i]))
# print(')')

In [223]:
regular19

,大会,チーム,選手名,試合数,セット数,アタック打数,アタック得点,アタック失点,アタック決定率,アタック効果率,...,サーブ効果,サーブ効果率,受数,サーブレシーブ成功・優,サーブレシーブ成功・良,サーブレシーブ成功率,アタック,ブロック,サーブ,得点合計
0,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,藤中優斗,29,79,152,64,12,42.1,34.2,...,41,4.1,287,120,85,56.6,64,14,5,83
8,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,金丸晃大,27,51,17,8,1,47.1,41.2,...,8,4.3,2,0,1,25.0,8,5,0,13
60,2019-20 V.LEAGUE DIVISION1 MEN,東レアローズ,伏見大和,56,86,135,84,7,62.2,57.0,...,27,3.2,5,3,2,80.0,84,36,9,129
89,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,福山汰一,29,47,94,55,2,58.5,56.4,...,30,6.0,8,3,3,56.2,55,21,9,85
108,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,饒書涵,27,96,296,177,7,59.8,57.4,...,84,6.9,25,12,7,62.0,177,40,7,224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2910,2019-20 V.LEAGUE DIVISION1 MEN,VC長野トライデンツ,森崎健史,27,91,217,112,19,51.6,42.9,...,62,3.9,53,28,11,63.2,112,30,11,153
2922,2019-20 V.LEAGUE DIVISION1 MEN,VC長野トライデンツ,松村文斗,24,3,10,1,2,10.0,-10.0,...,1,-10.0,21,8,7,54.8,1,0,0,1
2930,2019-20 V.LEAGUE DIVISION1 MEN,VC長野トライデンツ,大沢健太郎,22,17,0,0,0,35.6,NaN,...,0,3.0,19,4,10,47.4,0,0,0,0
2936,2019-20 V.LEAGUE DIVISION1 MEN,VC長野トライデンツ,高澤和貴,27,62,14,8,1,57.1,50.0,...,6,9.2,67,30,19,59.0,8,0,4,12


In [150]:
regular19.iloc[]

大会             2019-20 V.LEAGUE DIVISION1 MEN
チーム                                VC長野トライデンツ
選手名                                       長田翼
試合数                                        27
セット数                                       33
アタック打数                                      1
アタック得点                                      0
アタック失点                                      1
アタック決定率                                     0
アタック効果率                                  -100
バックアタック打数                                   0
バックアタック得点                                   0
バックアタック失点                                   0
バックアタック決定率                                  0
バックアタック効果率                                NaN
アタック得点セット平均                               0.7
ブロック得点                                      3
ブロック得点セット平均                              0.09
サーブ打数                                      44
ノータッチ                                       0
エース                                         1
サーブ失点                             

In [227]:
regular19.loc[-1] = [
     ob_data['大会'][0],
     None,
     '合計',
     regular19['試合数'].sum(),
     regular19['セット数'].sum(),
     regular19['アタック打数'].sum(),
     regular19['アタック得点'].sum(),
     regular19['アタック失点'].sum(),
     attack_ave.round(1),
     attack_ef.round(1),
     regular19['バックアタック打数'].sum(),
     regular19['バックアタック得点'].sum(),
     regular19['バックアタック失点'].sum(),
     b_attack_ave.round(1),
     b_attack_ef.round(1),
     attack_by_set.round(1),
     regular19['ブロック得点'].sum(),
     block_by_set.round(1),
     regular19['サーブ打数'].sum(),
     regular19['ノータッチ'].sum(),
     regular19['エース'].sum(),
     regular19['サーブ失点'].sum(),
     regular19['サーブ効果'].sum(),
     serve_ef.round(1),
     regular19['受数'].sum(),
     regular19['サーブレシーブ成功・優'].sum(),
     regular19['サーブレシーブ成功・良'].sum(),
     s_receive_ave.round(1),
     regular19['アタック'].sum(),
     regular19['ブロック'].sum(),
     regular19['サーブ'].sum(),
     regular19['得点合計'].sum()
]

In [158]:
average = regular19[regular19['選手名']=='合計']
average

,大会,チーム,選手名,試合数,セット数,アタック打数,アタック得点,アタック失点,アタック決定率,アタック効果率,...,サーブ効果,サーブ効果率,受数,サーブレシーブ成功・優,サーブレシーブ成功・良,サーブレシーブ成功率,アタック,ブロック,サーブ,得点合計
-2,2019-20 V.LEAGUE DIVISION1 MEN,None,合計,3501,8357,21531,10588,1313,49.2,43.1,...,4564,6.3,16224,6823,4180,54.9,10588,1804,805,13197


In [230]:
regular19_s = regular19[regular19['バックアタック打数'] >= 100]
regular19_s = regular19_s[['チーム','選手名','バックアタック打数','バックアタック得点','バックアタック失点','バックアタック決定率','バックアタック効果率']]
regular19_s.sort_values(by=['バックアタック決定率'], ignore_index=True).iloc[::-1]

,チーム,選手名,バックアタック打数,バックアタック得点,バックアタック失点,バックアタック決定率,バックアタック効果率
11,堺ブレイザーズ,千々木駿介,126,81,9,64.3,57.1
10,サントリーサンバーズ,ムセルスキードミトリー,406,240,33,59.1,51.0
9,サントリーサンバーズ,季道帥,135,76,9,56.3,49.6
8,ジェイテクトSTINGS,西田有志,437,243,48,55.6,44.6
7,堺ブレイザーズ,樋口裕希,119,66,4,55.5,52.1
6,大分三好ヴァイセアドラー,バグナスブライアン,174,90,17,51.7,42.0
5,パナソニックパンサーズ,清水邦広,210,107,12,51,45.2
4,None,合計,7858,3958,658,50.4,42.0
3,ウルフドッグス名古屋,高松卓矢,154,76,4,49.4,46.8
2,FC東京,プレモビッチピーター,444,215,29,48.4,41.9


In [232]:
regular19_stand = regular19.loc[:-1,regular19.columns.str.contains('決定率')].apply(lambda x : ((x - x.mean())*10/x.std()+50),axis=0)
regular19_stand

,アタック決定率,バックアタック決定率
0,46.443735,54.8893
8,49.871353,54.8893
60,60.222759,75.1756
89,57.686322,75.1756
108,58.577503,75.1756
...,...,...
2922,24.438427,49.2091
2930,41.987832,39.6745
2936,56.726589,39.6745
2944,17.583191,34.6029


In [237]:
regular19.loc[]

,大会,チーム,選手名,試合数,セット数,アタック打数,アタック得点,アタック失点,アタック決定率,アタック効果率,...,サーブ効果,サーブ効果率,受数,サーブレシーブ成功・優,サーブレシーブ成功・良,サーブレシーブ成功率,アタック,ブロック,サーブ,得点合計
0,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,藤中優斗,29,79,152,64,12,42.1,34.2,...,41,4.1,287,120,85,56.6,64,14,5,83
8,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,金丸晃大,27,51,17,8,1,47.1,41.2,...,8,4.3,2,0,1,25.0,8,5,0,13
60,2019-20 V.LEAGUE DIVISION1 MEN,東レアローズ,伏見大和,56,86,135,84,7,62.2,57.0,...,27,3.2,5,3,2,80.0,84,36,9,129
89,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,福山汰一,29,47,94,55,2,58.5,56.4,...,30,6.0,8,3,3,56.2,55,21,9,85
108,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,饒書涵,27,96,296,177,7,59.8,57.4,...,84,6.9,25,12,7,62.0,177,40,7,224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2910,2019-20 V.LEAGUE DIVISION1 MEN,VC長野トライデンツ,森崎健史,27,91,217,112,19,51.6,42.9,...,62,3.9,53,28,11,63.2,112,30,11,153
2922,2019-20 V.LEAGUE DIVISION1 MEN,VC長野トライデンツ,松村文斗,24,3,10,1,2,10.0,-10.0,...,1,-10.0,21,8,7,54.8,1,0,0,1
2930,2019-20 V.LEAGUE DIVISION1 MEN,VC長野トライデンツ,大沢健太郎,22,17,0,0,0,35.6,NaN,...,0,3.0,19,4,10,47.4,0,0,0,0
2936,2019-20 V.LEAGUE DIVISION1 MEN,VC長野トライデンツ,高澤和貴,27,62,14,8,1,57.1,50.0,...,6,9.2,67,30,19,59.0,8,0,4,12
